In [439]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np
from pathlib import Path

In [440]:
__ROOT_PATH__ = Path(__name__).resolve().parent.parent
input_path = __ROOT_PATH__ / "data/RotRolledCoid_Input.xlsx"
output_path = __ROOT_PATH__ / "export/bestPurchaseScenario.xlsx"

In [441]:
df_demand = pd.read_excel(input_path, sheet_name="DEMAND")
df_offer = pd.read_excel(input_path, sheet_name="OFFER")
df_index = pd.read_excel(input_path, sheet_name="INDEX")
df_fee = pd.read_excel(input_path, sheet_name="FEE")
df_freight = pd.read_excel(input_path, sheet_name="FREIGHT")

# Dropping
df_demand.drop(["Unit"], inplace=True, axis=1)

## Dataset Treamentments

In [442]:
df_demand = df_demand.melt(
    id_vars=["Rolled_Coil_Dim", "Factory"], 
    var_name="Period", 
    value_name="Demand"
    )

In [443]:
df_offer = df_offer.melt(
    id_vars=["Rolled_Coil_Dim", "Suppliers"], 
    var_name="Period", 
    value_name="Offer"
    )

In [444]:
df_freight = df_freight.melt(
    id_vars=["Supplier", "Factory"], 
    var_name="Freight_Type", 
    value_name="Freight_Price")

## Building Model

In [445]:
model = pyo.ConcreteModel()

## Defining Sets

In [446]:
period = df_demand["Period"].unique()
factory = df_demand["Factory"].unique()
freight = df_freight["Freight_Type"].unique()
supplier = df_offer["Suppliers"].unique()
dimension = df_demand["Rolled_Coil_Dim"].unique()

In [447]:
model.period = pyo.Set(initialize=period)
model.factory = pyo.Set(initialize=factory)
model.freight_type = pyo.Set(initialize=freight)
model.supplier = pyo.Set(initialize=supplier)
model.dimension = pyo.Set(initialize=dimension)

## Defining Parameters

In [448]:
demand_param = df_demand.set_index(["Rolled_Coil_Dim", "Factory", "Period"]).to_dict()["Demand"]
offer_param = df_offer.set_index(["Rolled_Coil_Dim", "Suppliers", "Period"]).to_dict()["Offer"]
commodity_price = df_index.set_index(["Period"]).to_dict()["Rolled_Coil_Index"]
usd_brl = df_index.set_index(["Period"]).to_dict()["Dolar_Value"]
fee = df_fee.set_index(["Rolled_Coil_Dim", "Supplier"]).to_dict()["Fee"]
freight = df_freight.set_index(["Supplier", "Factory", "Freight_Type"]).to_dict()["Freight_Price"]

In [449]:
model.demand = pyo.Param(model.dimension, model.factory, model.period, initialize=demand_param)
model.offer = pyo.Param(model.dimension, model.supplier, model.period, initialize=offer_param)
model.commodity_price = pyo.Param(model.period, initialize=commodity_price)
model.usd_brl = pyo.Param(model.period, initialize=usd_brl)
model.fee = pyo.Param(model.dimension, model.supplier, initialize=fee)
model.freight = pyo.Param(model.supplier, model.factory, model.freight_type, initialize=freight)

## Defining Variables

In [450]:
model.x = pyo.Var(model.dimension, model.factory, model.period, model.supplier, within=pyo.NonNegativeReals)
model.binDecisionSupplier = pyo.Var(model.dimension, model.factory, model.period, model.supplier, within=pyo.Binary)
model.artifitial = pyo.Var(model.dimension, model.factory, model.period, model.supplier, within=pyo.NonNegativeReals)

## Building Constraints

In [451]:
def demandSupply(model, d, f, p):
    return sum(
        model.x[d, f, p, s]
        for s in model.supplier
        ) >= model.demand[d, f, p]

model.demandSupply = pyo.Constraint(
    model.dimension, 
    model.factory, 
    model.period, 
    rule=demandSupply
    )

In [452]:
def offerLimit(model, d, f, p, s):
    return model.x[d, f, p, s] <= model.offer(d, s, p)

model.offerLimit = pyo.Constraint(
    model.dimension,
    model.factory,
    model.period,
    model.supplier
    )

In [453]:
def decisionSupply(model, d, f, p, s):
    bigM = model.demand[d, f, p]
    return model.x[d, f, p, s] <= model.binDecisionSupplier[d, f, p, s] * bigM

model.decisionSupply = pyo.Constraint(
    model.dimension,
    model.factory,
    model.period,
    model.supplier,
    rule=decisionSupply
)

## Building Objective Function

In [454]:
def getPrice(model, p):
    commodity_price = model.commodity_price[p]
    dolar = model.usd_brl[p]
    index_price = commodity_price * dolar
    return index_price

def getFee(model, d, s):
    fee = model.fee[d, s]
    return fee

def getFreight(model):
    INFINITY = 10000000
    for s in model.supplier:
        for f in model.factory:
            for t in model.freight_type:
                if model.freight[s, f, t] > 0:
                    return model.freight[s, f, t]
                else:
                    return INFINITY

In [455]:
def makePrice(model, d, s, p):
    index_price = getPrice(model, p)
    fee = getFee(model, d, s)
    freight = getFreight(model)
    value = ((index_price * fee) + index_price) + freight
    return value

### Objetive Function

In [456]:
def objetiveFunction(model):
    return sum(
        (model.x[d, f, p ,s] * makePrice(model, d, s, p)) + (model.artifitial[d, f, p ,s] * 100000)
        for d in model.dimension
        for f in model.factory
        for p in model.period
        for s in model.supplier
        )

model.objetiveFunction = pyo.Objective(
    rule=objetiveFunction, sense=pyo.minimize
    )

## Solving Model

In [457]:
solver = pyo.SolverFactory("glpk")
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 652513929.404497, 'Upper bound': 652513929.404497, 'Number of objectives': 1, 'Number of constraints': 720, 'Number of variables': 1728, 'Number of nonzeros': 1728, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.0674598217010498}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [458]:
output_generic = [
    [i[0], i[1], i[2], i[3], 
    model.x[i].value,model.commodity_price[i[2]], 
    model.usd_brl[i[2]], model.fee[i[0], i[3]]]
    for i in model.x
    if model.x[i].value > 0
]

In [459]:
columns = ["Dimension", "Factory", "Period", 
 "Supplier", 'Quantity', "Price_Commodity", 
 "Dolar", "Fee"]

intermediate_df = pd.DataFrame(output_generic, columns=columns)

In [460]:
freight_values_list = list()
freight_type_list = list()

for i in range(len(intermediate_df)):
    supplier = intermediate_df.loc[i, "Supplier"]
    factory = intermediate_df.loc[i, "Factory"]

    freight_price = df_freight[
        (df_freight["Supplier"]==supplier) 
        & (df_freight["Factory"]==factory) 
        & (df_freight["Freight_Price"] > 0)]["Freight_Price"].values[0]
    freight_type = df_freight[
        (df_freight["Supplier"]==supplier) 
        & (df_freight["Factory"]==factory) 
        & (df_freight["Freight_Price"] > 0)]["Freight_Type"].values[0]
    freight_values_list.append(freight_price)
    freight_type_list.append(freight_type)

In [461]:
output_df = intermediate_df.copy()
output_df["Freight_type"] = freight_type_list
output_df["Freight_price"] = freight_values_list

In [462]:
with pd.ExcelWriter(output_path) as writer:
    output_df.to_excel(
        excel_writer=writer,
        sheet_name="Purchase_Scenario",
        index = False
    )